In [18]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
import pickle
from lightgbm import LGBMClassifier
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, RobustScaler
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from typing import List
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV
from sklearn.utils.class_weight import compute_sample_weight, compute_class_weight
from imblearn.under_sampling import RandomUnderSampler 
import collections
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler

In [2]:
df_val = pd.read_csv('Validation-dataset.csv')
df = pd.read_csv('Train-dataset.csv')

In [20]:
encoder = {}
decoder = {}
for i, code in enumerate(df['LITH_CODE'].unique()):
    decoder[i] = code
    encoder[code] = i

In [21]:
feature = ['MD','GR', 'RT', 'DEN', 'DEPOSITIONAL_ENVIRONMENT']
numeric_features = ['MD','GR', 'RT', 'DEN', 'CN']
categorical_features = ['DEPOSITIONAL_ENVIRONMENT']
df_scale = df[feature]
df_val_scale = df[feature]
y = df['LITH_CODE'].apply(lambda x: encoder[x])

In [22]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self,attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [24]:
class ScaleAndEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, numeric):
        self.numeric = numeric

    def fit(self, X, y=None):
        self.scaler = StandardScaler()
        self.scaled = self.scaler.fit(X[self.numeric])
        return self

    def transform(self, X, y=None):
        self.scaled = self.scaler.transform(X[self.numeric])
        self.numeric_df = pd.DataFrame(data=self.scaled,
                                       columns=self.numeric)
        return self.numeric_df

In [25]:
numeric_features = ['MD','GR', 'RT', 'DEN']
categorical_features = ['DEPOSITIONAL_ENVIRONMENT']
pipe = Pipeline([('selector', ScaleAndEncoder(numeric_features))])

In [26]:
df = df.join(pd.get_dummies(df['DEPOSITIONAL_ENVIRONMENT']))

In [29]:
df_val = df_val.join(pd.get_dummies(df_val['DEPOSITIONAL_ENVIRONMENT']))

In [27]:
df.head(1)

,WELL,X,Y,MD,GR,RT,CN,DEN,DEPOSITIONAL_ENVIRONMENT,LITH_NAME,LITH_CODE,Continental,Marine,Transitional
0,Well-1,0.8179,2.9814,1602.0,83.939,3.166,0.25,2.344,Marine,Marly sandstone,1200,0,1,0


In [30]:
df_val.head(1)

,WELL,X,Y,MD,GR,RT,CN,DEN,DEPOSITIONAL_ENVIRONMENT,Continental,Marine,Transitional
0,Well-X,2.9956,0.0061,897.065,92.7094,4.51768,0.36366,2.2106,Continental,1,0,0


In [28]:
X = pipe.fit(df).transform(df)
X_test = pipe.transform(df_val)

In [32]:
X = X.join(df['CN'])
X = X.join(df['Continental'])
X = X.join(df['Marine'])
X = X.join(df['Transitional'])

X_test = X_test.join(df['CN'])
X_test = X_test.join(df['Continental'])
X_test = X_test.join(df['Marine'])
X_test = X_test.join(df['Transitional'])

In [34]:
X.sample()

,MD,GR,RT,DEN,CN,Continental,Marine,Transitional
34700,-0.306448,0.216787,-0.002083,0.515188,0.30207,1,0,0


In [35]:
X_test.sample()

,MD,GR,RT,DEN,CN,Continental,Marine,Transitional
1650,-0.587173,-0.054242,0.001093,0.002698,0.225,0,1,0


In [36]:
y = df['LITH_CODE'].apply(lambda x: encoder[x])
y[:5]

0    0
1    0
2    0
3    1
4    1
Name: LITH_CODE, dtype: int64

In [37]:
rf = classifier = xgb.XGBClassifier(
        eta=0.01,
        min_child_weight=2,
        max_depth=10,
        subsample=0.8,
        colsample_bytree=1,
        objective='auc'
)

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2)

cv_scores = cross_val_score(rf, X_train, y_train, cv=5)

In [38]:
cv_scores

array([0.91625683, 0.91871585, 0.92172131, 0.92540984, 0.91706517])

In [40]:
rf.fit(X_train, y_train)
f1 = f1_score(y_train, rf.predict(X_train), average='weighted')
print("F1 train score: %.3f" % f1)
f1 = f1_score(y_holdout, rf.predict(X_holdout), average='weighted')
print("F1 holdout score: %.3f" % f1)

F1 train score: 0.990
F1 holdout score: 0.928


In [41]:
# Fit the whole dataset
rf.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=10,
              min_child_weight=2, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.8, verbosity=1)

In [42]:
preds = rf.predict(X_test)

In [43]:
preds

array([7, 7, 5, ..., 2, 2, 2])

In [44]:
submission = []
for pred in preds:
    submission.append(decoder[pred])

In [47]:
submission

[800,
 800,
 100,
 100,
 800,
 800,
 800,
 800,
 800,
 800,
 800,
 100,
 100,
 600,
 600,
 500,
 500,
 1000,
 1000,
 1000,
 1000,
 500,
 500,
 500,
 500,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 500,
 600,
 600,
 600,
 1000,
 100,
 1100,
 100,
 100,
 100,
 800,
 500,
 500,
 800,
 800,
 500,
 500,
 800,
 800,
 800,
 800,
 100,
 100,
 100,
 100,
 100,
 1100,
 1100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 1100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 1000,
 100,
 100,
 100,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 1000,
 100,
 100,
 100,
 100,
 100,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 500,
 100,
 1000,
 100,
 100,
 1000,
 100,
 100,
 100,
 1000,
 1000,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 100,
 1000,
 1000,
 100,
 100,
 1000,
 1000,
 100,
 500,
 500,
 500,
 500,
 500,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 600,
 50

In [48]:
np.savetxt('prediction_xgb_final.csv', submission, delimiter=',', encoding='utf-8') 